In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pylab as pl
import matplotlib.animation as animation
# This is how we can import src module from sibling module
import os, sys
nb_dir = os.path.split(os.getcwd())[0]
print nb_dir
if nb_dir not in sys.path: sys.path.append(nb_dir)
#######################
from src import wdata
from netCDF4 import Dataset
import seaborn as sns;
from src.wdata import Sat, Radar
from skimage.transform import resize
from sklearn.utils import shuffle
from random import random as random 
import scipy.misc

C:\Users\Sa\Desktop\capstone-weather


In [2]:
def div2(arr):
    a1 = arr[:,:100]
    a2 = arr[:,100:]
    return (a1, a2)

def normBand(arr): # normalize and remap to [-1, 1]
    arr = arr - np.min(arr)
    arr = arr / np.max(arr)
    return (arr-0.5)*2

In [3]:
# todo: add code to automatically create directory structure where images will be saved
savedir = os.path.abspath("E:/weatherFall/run5")
radar_ = [i for mo in [7,8,9] for i in Radar.getMonth(2017,mo)]
bands, radar = [], []
sparsecount = 0
for tup in radar_:
    rad = np.flip(tup[1],0)[:100,:200] # crop to 126x200
    # get corrosponding Satelite image
    ds2 = Sat.getSatFromRad(tup, 2)
    ds4 = Sat.getSatFromRad(tup, 4)
    ds6 = Sat.getSatFromRad(tup, 6)
    
    if ds2 and ds4 and ds6: 
        b2 = div2(resize(normBand(Sat.getAndClose(ds2)), (126,201))[:100,:200])
        b4 = div2(resize(normBand(Sat.getAndClose(ds4)), (126,201))[:100,:200])
        b6 = div2(resize(normBand(Sat.getAndClose(ds6)), (126,201))[:100,:200])
        rad1, rad2 = div2(rad)
        
        if Radar.isSparse(rad1, threshold=0.00175): 
            if random() < 0.05:               
                radar.append(rad1)
                bands.append(np.stack((b2[0],b4[0],b6[0]), axis=2))
                sparsecount += 1
        else:
            radar.append(rad1)
            bands.append(np.stack((b2[0],b4[0],b6[0]), axis=2))
        
        if Radar.isSparse(rad2, threshold=0.00175): 
            if random() < 0.05:    
                radar.append(rad2)
                bands.append(np.stack((b2[1],b4[1],b6[1]), axis=2))
                sparsecount += 1
        else:
            radar.append(rad2)
            bands.append(np.stack((b2[1],b4[1],b6[1]), axis=2))
            
#         if Radar.isSparse(rad3, threshold=0.00175): 
#             if random() < 0.05:    
#                 radar.append(rad3)
#                 bands.append(np.stack((b2[2],b4[2],b6[2]), axis=2))
#                 sparsecount += 1
#         else:
#             radar.append(rad3)
#             bands.append(np.stack((b2[2],b4[2],b6[2]), axis=2))

bands, radar = shuffle(bands, radar)

d:\Anaconda2\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [4]:
size = len(bands)
print size
print len(bands), len(radar)
print 'Num of sparse images:', sparsecount

417
417 417
Num of sparse images: 34


In [5]:
splitratio = 0.80 # train percentage
def saveImgs():
    # save train set
    for i,d in enumerate(bands[:int(size*splitratio)]):
        scipy.misc.imsave(savedir+"/train/A/{:>03d}A{:d}.png".format(i,0), d[...,0])
        scipy.misc.imsave(savedir+"/train/A/{:>03d}A{:d}.png".format(i,1), d[...,1])
        scipy.misc.imsave(savedir+"/train/A/{:>03d}A{:d}.png".format(i,2), d[...,2])
    for i,d in enumerate(radar[:int(size*splitratio)]):
        scipy.misc.imsave(savedir+"/train/B/{:>03d}.png".format(i), np.flip(d*255,1).astype(np.uint8))
    
    # save val set
    for i,d in enumerate(bands[int(size*splitratio):]):
        scipy.misc.imsave(savedir+"/val/A/{:>03d}A{:d}.png".format(i,0), d[...,0])
        scipy.misc.imsave(savedir+"/val/A/{:>03d}A{:d}.png".format(i,1), d[...,1])
        scipy.misc.imsave(savedir+"/val/A/{:>03d}A{:d}.png".format(i,2), d[...,2])
    for i,d in enumerate(radar[int(size*splitratio):]):
        scipy.misc.imsave(savedir+"/val/B/{:>03d}.png".format(i),(d*255).astype(np.uint8))
    print('train, val count: {:d}, {:d}'.format(int(size*splitratio), size-int(size*splitratio)))
saveImgs()

d:\Anaconda2\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  """
d:\Anaconda2\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  
d:\Anaconda2\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  import sys
d:\Anaconda2\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  if __name__ == '__main__':
d:\Anaconda2\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, a

train, val count: 333, 84


d:\Anaconda2\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
